# Data

In [1]:
data = [
    1 2 1 10
    1 3 10 3
    2 4 1 1 
    2 5 2 3 
    3 2 1 2
    3 4 5 7
    3 5 12 3
    4 5 10 1
    4 6 1 7
    5 6 2 2
]
start_node = data[:, 1]
end_node = data[:, 2]
arc_cost = data[:, 3]
arc_time = data[:, 4]

origin, destination = 1, 6

n_nodes = 6
n_arcs = length(start_node)

10

![](https://i.imgur.com/ZMV4jtO.png)

# The Restricted Master Problem

In [2]:
# Restricted Master Problem function을 만들고 싶어. 해당 function은 그 결과로 pi_1, pi_0을 return해야해. 
using JuMP, HiGHS

path1 = [1, 2, 4, 5, 6]
path2 = [1, 2, 4, 6]
# path3 = [1, 3, 2, 5, 6]
# path4 = [1, 2, 5, 6]

paths = [path1, path2]
Cost = [14, 3]
Time = [14, 18]

function RestrictedMaster(paths, Cost, Time)
    m = Model(HiGHS.Optimizer)
    @variable(m, λ[1:length(paths)] >= 0) # relaxation
    @objective(m, Min, sum(Cost[i] * λ[i] for i in 1:length(paths)))
    @constraint(m, resource, sum(Time[i] * λ[i] for i in 1:length(paths)) <= 14)
    @constraint(m, convexity, sum(λ[i] for i in 1:length(paths)) == 1)
    optimize!(m)
    π1 = dual(resource)
    π0 = dual(convexity)
    return m, λ, π1, π0
end

m, λ, π1, π0 = RestrictedMaster(paths, Cost, Time)
@show raw_status(m)
@show objective_value(m)
@show value.(λ)
@show π1, π0;

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+01]
  Cost   [3e+00, 1e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+01]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-2); columns 0(-2); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.4000000000e+01
HiGHS run time      :          0.00
raw_status(m) = "kHighsModelStatusOptimal"
objective_value(m) = 14.0
value.(λ) = [1.0, -0.0]
(π1, π0) = (-2.75, 52.5)


# Solving the Pricing Sub-Problem

In [3]:
# 예시 코드
using Graphs, SimpleWeightedGraphs
g = SimpleWeightedDiGraph(n_nodes)
for i in 1:n_arcs
    # if (start_node[i], end_node[i]) != (1, 2)
    add_edge!(g, start_node[i], end_node[i], arc_cost[i] - arc_time[i] * π1) # cost redifined
    # end
end
bf_state = bellman_ford_shortest_paths(g, origin)
@show bf_state.dists
@show bf_state.dists[destination]
min_reduced_cost = bf_state.dists[destination] - π0
new_column = enumerate_paths(bf_state)[destination]
@show min_reduced_cost
@show new_column;

bf_state.dists = [0.0, 24.75, 18.25, 28.5, 35.0, 42.5]
bf_state.dists[destination] = 42.5
min_reduced_cost = -10.0
new_column = [1, 3, 2, 5, 6]


In [1]:
# solving RMP + solving pricing sub-priblem = column generation이야. 그러니깐 난 column generation function을 만들고 싶어.
# 위에서 얻어진 pi_1, pi_0을 이용하여 Pricing Subproblem을 풀고 싶어
# min_reduced_cost와 new_column을 이용하여 Pricing Subproblem을 풀고, 새로운 path를 얻고 싶어 
# 이때, min_reduced_cost가 0보다 작아야 새로운 path를 Restricted Master Problem에 추가해야해. 왜냐하면 그래야 더 좋은 path가 있다는 것이니깐
# 해당 function은 Restricted Master Problem을 input으로 받아서 path를 추가한 후, 업데이트된 Restricted Master Problem을 return해야해
# 추가로, path를 추가하는 function을 이용하여 Restricted Master Problem을 풀고 싶어
# 이를 통해 최종적으로 Restricted Master Problem을 풀고, 최적해를 얻고 싶어


using Graphs, SimpleWeightedGraphs

function PricingSubproblem(π1, π0, origin, destination, arc_cost, arc_time)
    g = SimpleWeightedDiGraph(n_nodes)
    for i in 1:n_arcs
        add_edge!(g, start_node[i], end_node[i], arc_cost[i] - arc_time[i] * π1) # cost redifined
    end
    bf_state = bellman_ford_shortest_paths(g, origin)
    min_reduced_cost = bf_state.dists[destination] - π0
    new_column = enumerate_paths(bf_state)[destination]
    return min_reduced_cost, new_column
end

function ColumnGeneration(m, π1, π0, origin, destination, arc_cost, arc_time)
    min_reduced_cost, new_column = PricingSubproblem(π1, π0, origin, destination, arc_cost, arc_time)
    if min_reduced_cost < 0
        push!(paths, new_column)
        @show paths
        push!(Cost, sum(arc_cost[i] for i in new_column))
        push!(Time, sum(arc_time[i] for i in new_column))
        m, λ, π1, π0 = RestrictedMaster(paths, Cost, Time)
        return ColumnGeneration(m, π1, π0, origin, destination, arc_cost, arc_time)
    else
        return m
    end
end

m = ColumnGeneration(m, π1, π0, origin, destination, arc_cost, arc_time)
@show raw_status(m)
@show objective_value(m)
@show value.(λ)
@show π1, π0;

UndefVarError: UndefVarError: `m` not defined

In [ ]:
include("data.jl")
include("column_generation.jl")
using DataStructures
# 현재 ColumnGeneration code를 구현해두었어. 하지만 나는 해당 코드를 이용한 branch-and-bound algorithm을 구현하고 싶어.
# 이를 위해서는 ColumnGeneration 코드를 수정해도 괜찮아. 특히 branch-and-bound algorithm에서 braching을 할때, branching variable이 0 또는 1 중에 선택되면 이를 반영하여 RestrictedMaster를 풀어야 하기 때문에 이를 고려해서 코드를 수정해야 할거야.
# 매 branching마다 ColumnGeneration을 다시 반복해서 풀어야겠지?
# branching variable 선택 방식은 다음과 같아. ColumnGeneration을 풀고 나온 최종 결과의 각 path의 λ 값들 중 가장 높은 λ 값을 가지는 path중 하나의 arc를 branching variable로 선택할거야.
# 이를 통해 branching variable이 선택되면 해당 arc가 있는 path만 골라내서 다시 RestrictedMaster를 풀어야 해.

# branch-and-bound는 다음 3가지 과정으로 이루어져 있어. 1) branching, 2) bounding, 3) fathoming
# Fathoming test는 다음과 같아. A subproblm is fathomed (dismissed from further consideration) if Test1) Its bound >= Z* or Test2) Its LP relaxation has no feasible solutions or Test3) The optimal solution for its LP relaxation is integer.(If this solution is better than the incumbent, it becomes the new incumbent, and test1 is reapplied to all unfathomed subproblems with the new smaller Z*)
# Initialization : Set Z* = ∞. If not fathomed, clsssify this problem as the one remaining "subproblem" for performing the first full iteration below.
# Branching : Among the remaining (unfathomed) subproblems, select the one that was created most recently. Branch from the node for this subproblem to create two new subproblems by fixing the next variable (the branching variable) at either 0 or 1.
# Bounding : For each new subproblem, solve its LP relaxation to obtain an optimal solution. If this value of Z is not an integer, round it up to an integer (If it was already an integer, no change is needed.) This integer value of Z is the bound for the subproblem.
# Fathoming : For each new subproblem, apply the three fathoming tests summarized above, and discard those subproblems that are fathomed by any of the tests.
# Optimality test : Stop when there are no remaining subproblems that have not been fathomed. The current incumbent is optimal. Otherwise, return to perform another iteration.
# 이를 통해 branch-and-bound algorithm을 julia로 구현해줘.
# branch-and-bound algorithm은 tree 구조를 가져야해. 그래야 fathomed되었을때, 더 이상 해당 branch를 진행하지 않고 parent node를 backtracking하여 다른 branch를 진행할 수 있어. 이를 고려해서 subproblems를 저장할 수 있는 자료구조를 사용해야 해.

struct TreeNode
    data::Any
    parent::Union{TreeNode, Nothing}
    children::Vector{TreeNode}
end

function add_child(parent::TreeNode, data::Any)
    child = TreeNode(data, parent, [])
    push!(parent.children, child)
    return child
end

function backtrack(node::TreeNode)
    return node.parent
end

function is_fathomed(m, Z_star)
    # Test 1: Its bound >= Z*
    if objective_value(m) >= Z_star
        return true
    end

    # Test 2: Its LP relaxation has no feasible solutions
    if termination_status(m) != MOI.OPTIMAL
        return true
    end

    # Test 3: The optimal solution for its LP relaxation is integer
    if objective_value(m) == round(objective_value(m))
        return true
    end

    return false
end

function BranchAndBound(origin, destination, arc_cost, arc_time, paths, Cost, Time)
    # Initialization
    Z_star = Inf
    m, λ, π1, π0 = ColumnGeneration(origin, destination, arc_cost, arc_time, paths, Cost, Time)
    root = TreeNode((m, λ, π1, π0, paths, Cost, Time), nothing, []) # Create a root node for the initial subproblem
    subproblems = Deque{TreeNode}() # Create an empty Deque of TreeNodes
    push!(subproblems, root) # Push the root node into the deque
    # Branching
    while !isempty(subproblems)
        # Among the remaining (unfathomed) subproblems, select the one that was created most recently.
        node = popfirst!(subproblems)
        m, λ, π1, π0, paths, Cost, Time = node.data
        # Branch from the node for this subproblem to create two new subproblems by fixing the next variable (the branching variable) at either 0 or 1.
        @show value.(λ)
        max_λ_index = findmax(value.(λ))[2]
        @show max_λ_index
        path = paths[max_λ_index]
        @show paths
        @show path
        branching_variables = []
        for i in 1:n_arcs
            push!(branching_variables, [start_node[i], end_node[i]])
        end
        branching_variable = popfirst!(branching_variables)
        @show branching_variable
        first_indices = findall(path -> any(path[i:i+1] == branching_variable for i in 1:length(path)-1), paths)
        @show first_indices
        second_indices = setdiff(1:length(paths), first_indices)
        @show second_indices

        indices_dict = Dict(1 => first_indices, 2 => second_indices)
        for i in 1:2
            new_paths = copy(paths)
            new_Cost = copy(Cost)
            new_Time = copy(Time)
            i_indices = indices_dict[i]
            new_paths = new_paths[i_indices]
            new_Cost = new_Cost[i_indices]
            new_Time = new_Time[i_indices]
            @show new_paths
            @show new_Cost
            @show new_Time
            m, λ, π1, π0 = ColumnGeneration(origin, destination, arc_cost, arc_time, new_paths, new_Cost, new_Time)
            child = TreeNode((m, λ, π1, π0, new_paths, new_Cost, new_Time), node, []) # Create a new TreeNode for each child subproblem
            push!(subproblems, child)
        end

        # If the subproblem is not fathomed, enqueue its children
        if !is_fathomed(m, Z_star)
            children_data = generate_children(m, λ, π1, π0, paths, Cost, Time)
            for child_data in children_data
                child = TreeNode(child_data, node, []) # Create a new TreeNode for each child subproblem
                push!(subproblems, child)
            end
        end
        
        # Bounding
        for i in 1:2
            node = popfirst!(subproblems) # Remove and return the first element from the deque
            m, λ, π1, π0, new_paths, new_Cost, new_Time = node.data
            if termination_status(m) == MOI.OPTIMAL
                if objective_value(m) != round(objective_value(m))
                    set_objective_function(m, round(objective_value(m)))
                end
                if objective_value(m) < Z_star
                    Z_star = objective_value(m)
                end
            end
        end

        # Fathoming
        for i in 1:2
            node = popfirst!(subproblems) # Remove and return the first element from the deque
            m, λ, π1, π0, new_paths, new_Cost, new_Time = node.data
            if termination_status(m) == MOI.OPTIMAL
                if objective_value(m) >= Z_star
                    # No need to remove elements from the deque because we already took them out
                end
            end
        end
    end
    return Z_star
end

BranchAndBound(origin, destination, arc_cost, arc_time, paths, Cost, Time)
